<a href="https://colab.research.google.com/github/ambarishg/open-source-llms/blob/main/RAG_ZEPHYR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install qdrant-client sentence-transformers accelerate

In [ ]:
# Qdrant server URL
URL ="<QDRANT_URL>"
# Qdrant dimension of the collection
DIMENSION = 384
# Qdrant collection name
COLLECTION_NAME = "<COLLECTION_NAME>"
METRIC_NAME ="COSINE"
MODEL_NAME="all-MiniLM-L6-v2"

In [ ]:
import qdrant_client as qc
import qdrant_client.http.models as qmodels
from qdrant_client.http.models import *

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
import torch
from transformers import pipeline

In [ ]:
user_input = "Describe Platyhelminthes"
query_filter =  None

In [ ]:
def get_qdrant_client():
    client = qc.QdrantClient(url=URL)
    METRIC = qmodels.Distance.COSINE
    return client

In [ ]:
client = get_qdrant_client()

In [ ]:
client

In [ ]:
def load_model():
    return SentenceTransformer(MODEL_NAME)

In [ ]:
model = load_model()
xq = model.encode(user_input,convert_to_tensor=True)

In [ ]:
xq

In [ ]:
search_result = client.search(collection_name=COLLECTION_NAME,
                                    query_vector=xq.tolist(),
                                    query_filter=query_filter,
                                    limit=5)

In [ ]:
search_result

In [ ]:
contexts =""
for result in search_result:
    contexts +=  result.payload['token']+"\n---\n"

In [ ]:
contexts

In [ ]:
%%time
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1",
                                            torch_dtype = torch.bfloat16,
    device_map = "auto",
    trust_remote_code = True)
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1")
prompt = f"""Answer based on context:\n\n{contexts}\n\n{user_input}"""
messages = [
     {"role": "user", "content": prompt},
]

model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt")
generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])